In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import sys
import random
from itertools import product

from train import *

IM HERE
im here


In [2]:
data_pairs = pd.read_csv("./data/train_split.csv")
data_questions = pd.read_csv("./data/data_questions.csv")

In [3]:
all_data_pairs = pd.read_csv("./data/all_possible_pairs.csv")
qids_to_index = dict()

for index, (qid1, qid2, question1, question2, is_duplicate) in all_data_pairs.iterrows():
    qids_to_index[(qid1, qid2)] = index

def get_pair_index(qid1, qid2):
    if (qid1, qid2) in qids_to_index:
        return qids_to_index[(qid1, qid2)]
    return qids_to_index[(qid2, qid1)]

In [4]:
graph_similar = defaultdict(lambda: [])
original_similar_pairs = set()
for _, (qid1, qid2, _, _, is_duplicate, _) in data_pairs.iterrows():
    if not is_duplicate: continue

    graph_similar[qid1].append(qid2)
    graph_similar[qid2].append(qid1)

    lower_id = min(qid1, qid2)
    higher_id = max(qid1, qid2)
    original_similar_pairs.add((lower_id, higher_id))

graph_non_similar = defaultdict(lambda: [])
original_non_similar_pairs = set()
for _, (qid1, qid2, _, _, is_duplicate, _) in data_pairs.iterrows():
    if is_duplicate: continue

    graph_non_similar[qid1].append(qid2)
    graph_non_similar[qid2].append(qid1)

    lower_id = min(qid1, qid2)
    higher_id = max(qid1, qid2)
    original_non_similar_pairs.add((lower_id, higher_id))

In [5]:
labels = defaultdict(lambda : 0)
components = defaultdict(lambda: [])

def visit(node, level):
    stack = [node]
    while len(stack) > 0:
        v = stack.pop()
        if labels[v] != 0: continue
        labels[v] = level
        components[level].append(v)
        stack += graph_similar[v]

level = 0
for node in graph_similar:
    if labels[node] == 0:
        level += 1
        visit(node, level)

In [6]:
augmented_negative_pairs = set()

for qid1, qid2 in original_non_similar_pairs:
    k1 = labels[qid1]
    k2 = labels[qid2]
    if k1 == k2: continue

    k1_components = components[k1]
    k2_components = components[k2]
    for qid1_new, qid2_new in product(k1_components, k2_components):
        lower_id = min(qid1_new, qid2_new)
        higher_id = max(qid1_new, qid2_new)
        augmented_negative_pairs.add((lower_id, higher_id))

augmented_negative_pairs = augmented_negative_pairs.difference(original_non_similar_pairs)
augmented_negative_pairs = list(augmented_negative_pairs)

In [7]:
len(augmented_negative_pairs)

121263

In [9]:
pairs = []
for qid1, qid2 in augmented_negative_pairs:
    pairs.append([get_pair_index(qid1, qid2), 0])

augmented_nagative_pairs_dataset = pd.DataFrame(pairs, columns=["pair_id", "is_duplicate"])
augmented_nagative_pairs_dataset.to_csv("./data/negative_augmented_pairs.csv")

In [10]:
dataset = QQPDataset("./data/negative_augmented_pairs.csv", './embeddings/all_embeddings.npy')
loader =DataLoader(dataset, batch_size=32, shuffle=False)

In [12]:
model = QQPModel()
model.load_state_dict(torch.load("./checkpoints/model_2_1.00_7.pt"))
model.to("cuda:0")

QQPModel(
  (seq): Sequential(
    (0): Linear(in_features=1536, out_features=3072, bias=True)
    (1): BatchNorm1d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=3072, out_features=3072, bias=True)
    (4): BatchNorm1d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=3072, out_features=1, bias=True)
  )
)

In [30]:
model.eval()
all_predictions = []
with torch.no_grad():
    for inputs, labels in loader:
        inputs = inputs.to("cuda:0")
        outputs = model(inputs)
        predicted = torch.sigmoid(outputs)
        all_predictions += predicted.detach().cpu().numpy().tolist()

all_predictions = np.array(all_predictions).squeeze()
sorted_arg = np.argsort(all_predictions)

In [40]:
for index in sorted_arg[-100:]:
    pair_id = augmented_nagative_pairs_dataset.iloc[index].pair_id.item()
    pair = all_data_pairs.iloc[pair_id]
    print(pair.question1)
    print(pair.question2)
    print(all_predictions[index])
    print()

Why does Quora always marks my question as needing improvement?
Why does Quora mark my perfectly semantic question as, "Needs Improvement"?
0.9867807030677795

Why is Quora flagging so many Questions as needing improvement?
Why are so many questions about president elect Trump getting flagged as needing improvement when many are clear and concise?
0.9868947267532349

What are the safety precautions on handling shotguns proposed by the NRA in the entire U.S. including it’s territories and possessions?
What are the safety precautions on handling shotguns proposed by the NRA in Arkansas?
0.9869258403778076

Is it truly possible that we can travel back in time?
How can we travel back in time?
0.9871231317520142

How can I trace blocked calls?
Is there any app by which I can trace the exact location of a person by putting their phone number only?
0.9871518611907959

Is world War 3 currently happening?
Will there be WW3 soon? Why or why not?
0.987261176109314

Is Quora censored?
Why is there